# Our method

In [21]:
import numpy as np
import gurobipy as grb
from gurobipy import GRB
import random

## Simulated data

In [43]:
# obs: observed period: t = 1, 2, ... , obs
# first half data is generated based on Beta_1, while the others are based on Beta_2
obs = 200

In [44]:
np.random.seed(123)
mean = np.zeros(5)
cov = np.diag(np.ones(5))
X_const = np.ones(int(obs/2)).reshape((int(obs/2), 1))
X_1 = np.random.multivariate_normal(mean, cov, int(obs/2))
X_2 = np.random.multivariate_normal(mean, cov, int(obs/2))

X_1 = np.concatenate((X_const, X_1), axis = 1)
X_2 = np.concatenate((X_const, X_2), axis = 1)

Beta_1 = np.ones(6).reshape((6, 1))
Beta_2 = np.array([1, -1, 0, -1, 0, 1]).reshape((6, 1))

epsilon_1 = np.random.normal(loc=0, scale=1, size=int(obs/2)).reshape((int(obs/2), 1))
epsilon_2 = np.random.normal(loc=0, scale=1, size=int(obs/2)).reshape((int(obs/2), 1))

Y_1 = X_1@Beta_1 + epsilon_1
Y_2 = X_2@Beta_2 + epsilon_2

Y = np.concatenate((Y_1, Y_2), axis = 0)
X = np.concatenate((X_1, X_2), axis = 0)
epsilon =  np.concatenate((epsilon_1, epsilon_2), axis = 0)

In [45]:
print('Shape of Y: {} '.format(Y.shape))
print('Shape of X: {} '.format(X.shape))

Shape of Y: (200, 1) 
Shape of X: (200, 6) 


In [46]:
Beta_2

array([[ 1],
       [-1],
       [ 0],
       [-1],
       [ 0],
       [ 1]])

## Preparation for implementation

In [47]:
# Create true matrix for Beta
Beta_1_con = Beta_1
Beta_2_con = Beta_2

for i in range(int(obs/2)-1):
    Beta_1_con = np.concatenate((Beta_1_con, Beta_1), axis = 1)
    Beta_2_con = np.concatenate((Beta_2_con, Beta_2), axis = 1)

True_Beta = np.concatenate((Beta_1_con, Beta_2_con), axis = 1).T
True_Beta

array([[ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       ...,
       [ 1., -1.,  0., -1.,  0.,  1.],
       [ 1., -1.,  0., -1.,  0.,  1.],
       [ 1., -1.,  0., -1.,  0.,  1.]])

In [48]:
# True sum of squared residuals
resid = Y - np.sum(np.multiply(X, True_Beta), axis = 1).reshape((obs, 1))
ssr = np.sum(np.square(resid))
ssr

189.69794730645606

## Implementation

In [54]:
# create a model instance called Diet
M = grb.Model('MIOP')

time = 200
number_of_covariates = X.shape[1]

# set decision variables
beta = M.addMVar((time, 6), lb = -GRB.INFINITY) # change default setting lb = 0
Z = M.addMVar(time-1, vtype=GRB.BINARY)

# set objective function
Lambda  = 20
M.setObjective(grb.quicksum((Y[t]-X[t, :]@beta[t,:].T)*(Y[t]-X[t, :]@beta[t, :].T) for t in range(time)) + Lambda*grb.quicksum(Z))

# add constraints
# for i in range(time-1):
 #   M.addConstrs((Z[i] == 0) >> (beta[i, j] == beta[i+1, j]) for j in range(number_of_covariates))
    
# Big M
M_num = 100
for t in range(time-1):
    M.addConstrs( beta[t+1, p] - beta[t, p] <= M_num*Z[t] for p in range(number_of_covariates))
    M.addConstrs( beta[t+1, p] - beta[t, p] >= -1*M_num*Z[t] for p in range(number_of_covariates))
M.addConstr(Z.sum() == 1)   
M.optimize()

print('------------------------------------')
# value of objective function
print('objective function is minimised at: {:.2f}'.format(M.objVal))

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2389 rows, 1399 columns and 7363 nonzeros
Model fingerprint: 0x783330ab
Model has 4200 quadratic objective terms
Variable types: 1200 continuous, 199 integer (199 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e-04, 5e+01]
  QObjective range [1e-08, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1356.4329277
Presolve time: 0.02s
Presolved: 2389 rows, 1399 columns, 7363 nonzeros
Presolved model has 4200 quadratic objective terms
Variable types: 1200 continuous, 199 integer (199 binary)

Root relaxation: objective 2.000000e+01, 4063 iterations, 0.45 seconds (0.54 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth I

In [55]:
Z.X

array([-0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0.,
        0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0., -0., -0., -0.,
       -0.,  0.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0., -0., -0.,
       -0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,
       -0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,
       -0.,  0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0.,
       -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  1., -0., -0., -0., -0.,
       -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
       -0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,
        0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,
        0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0

In [56]:
np.where(Z.X == 1)

(array([99]),)

In [ ]:
test = np.array([1, 0, 1])
test2 = np.array([1, 0, 1])
np.allclose(test, test2)

In [ ]:
Z_list[1]

In [ ]:
len(Z)

In [ ]:
# compute the residual for each time
for i in 1:obs:
    resid_calc = Y[1] - 

In [ ]:
Y[1]


In [ ]:
# compute residulas using matrix
resid = Y - np.sum(np.multiply(X, True_Beta), axis = 1).reshape((obs, 1))
resid

In [ ]:
epsilon

In [ ]:
# residual is equivalent to epsilon
# Squared sum of squares should be:
resid = Y - np.sum(np.multiply(X, True_Beta), axis = 1).reshape((obs, 1))
SSR = np.sum(np.square(resid))
SSR

In [ ]:
# we can use l1 norm ?
np.linalg.norm(np.square(resid), ord=1)

## Implementation

In [ ]:
# create a model instance called Diet
M = grb.Model('MIOP')

# set decision variables
beta = M.addMVar((20, 5), lb = -100) # change default setting lb = 0
Z = M.addMVar((20, 5), vtype=GRB.BINARY)

# set objective function
Xbeta = np.sum(np.multiply(X, beta), axis = 1).reshape((20, 1))
M.setObjective(np.linalg.norm(np.square(Y - Xbeta), ord=1))

In [ ]:
range(6)

In [ ]:
# create a model instance called Diet
M = grb.Model('MIOP')

# set decision variables
x = M.addMVar(6)

# c^T
c = np.array([2,3.5,8,1.5,11,1])

# A
A = np.array([
    [-4,-8,-7,-1.3,-8,-9.2],
    [1,5,9,0.1,7,1],
    [15,11.7,0.4,22.6,0,17],
    [90,120,106,97,130,180],
    [0,0,0,0,1,0],
    [0,-1,0,0,0,0]
])

# b
b = np.array([-10,8,10,300,0.5,-1])

# inner products of C^T and x
M.setObjective(c@x)

# Ax >= b
M.addConstr(A@x >= b)

M.optimize()
print('------------------------------------')
# value of objective function
print('Cost is minimised at: {:.2f}'.format(M.objVal))

food = ['bread', 'milk', 'cheese','potato','fish','yogurt'] 

# where item comes from?
for tomo in range(len(food)):
    print(tomo)
    print('{:.2f} units of {} is included in the diet.'.format(x.x[tomo],food[tomo]))